## Milestone 1 - Plot the World

### Tasks: Load a world countries geojson
###        Display using folium
###        Add base location as point marker

In [ ]:
"""
**************************************************************************
*
* Author: Cooper Wolf
* Email: ctwolf1014@my.msutexas.edu
* Label: P01
* Title: Missile Geometry
* Course: CMPS 5993
* Semester: Spring 2026
*
* Description:
* This noteboook visualizes the geometry of a missile's range using Folium and GeoPandas.
* It displays a world map with country boundaries and marks the base location from which the missile is launched.
*
* 
*
* Files:
* Missile_Geometry.ipynb        : driver program
* countries.geo.json            : GeoJSON file containing world country boundaries
*
**************************************************************************
"""

In [ ]:
# Installing libraries
%pip install folium geopandas

### Load world geojson

In [39]:
# Importing libraries
import geopandas as gpd
import folium

# Reading the world data
world = gpd.read_file("countries.geojson")

# Creating a map
m = folium.Map(location=[20,0], zoom_start=2)

### Display using folium

In [40]:
# Converting GeoDataFrame to GeoJSON and adding to Folium
folium.GeoJson(world, name="World Countries").add_to(m)

### Adding base location as point marker

In [41]:
# Adding base location as a point marker
base_lat = 38.89767
base_lon = -77.03655

# Adding a marker for the base location
folium.Marker(location=[base_lat, base_lon], popup="Base", icon=folium.Icon(color="red", icon="home")).add_to(m)

In [ ]:
m

## Milestone 2 - Distance & Bearing

### Task: Compute distance from each threat origin to base
###       Identify closest threat
###       Display threat origin as points

#### Helper functions graciously provided by Dr. G

In [42]:
from __future__ import annotations
import math
from typing import Tuple


# ----------------------------------------
# Constants
# ----------------------------------------
EARTH_RADIUS_KM = 6371.0088  # mean Earth radius


# ----------------------------------------
# Distance: Haversine (km)
# ----------------------------------------
def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Compute great-circle distance between two lat/lon points.
    Returns distance in kilometers.
    """
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)

    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlmb / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return EARTH_RADIUS_KM * c


# ----------------------------------------
# Destination point (bearing + distance)
# ----------------------------------------
def destination_point(
    lat: float, lon: float, bearing_deg: float, distance_km: float
) -> Tuple[float, float]:
    """
    Compute destination lat/lon given:
      - start lat/lon
      - bearing in degrees
      - distance in km

    Uses great-circle formula.
    """
    bearing = math.radians(bearing_deg)

    lat1 = math.radians(lat)
    lon1 = math.radians(lon)
    d = distance_km / EARTH_RADIUS_KM

    lat2 = math.asin(
        math.sin(lat1) * math.cos(d) + math.cos(lat1) * math.sin(d) * math.cos(bearing)
    )

    lon2 = lon1 + math.atan2(
        math.sin(bearing) * math.sin(d) * math.cos(lat1),
        math.cos(d) - math.sin(lat1) * math.sin(lat2),
    )

    # normalize longitude to [-180, 180]
    lon2 = (math.degrees(lon2) + 540) % 360 - 180
    lat2 = math.degrees(lat2)

    return lat2, lon2

### Computing distance from each threat to base
### Identifing closest threats

In [ ]:
## Read in threats, calculate each distance from base, and identify closest threat, and display their original location as points
import json


## List to store threat ID and its distance from base
threat_dist = {}

# Read in threats from JSON file
with open("threats.json", "r") as f:
    threats = json.load(f)
    
# Calculate distance from base for each threat and store in dictionary
for threat in threats:
    threat_dist[threat["id"]] = haversine_km(base_lat, base_lon, threat["origin_lat"], threat["origin_lon"])
    
# Printing threat and its distance from base
print("Threat distances from base:")
for threat, distance in threat_dist.items():
    print(f"Threat ID: {threat}, Distance from base: {distance:.2f} km")


# Identify closest threat    
ordered_by_distance = sorted(threat_dist.items(), key=lambda x: x[1])
print("\nThreats ordered by distance from base:")
for threat, distance in ordered_by_distance:
    print(f"Threat ID: {threat}, Distance from base: {distance:.2f} km")


### Displaying threat origins as points

In [44]:
from folium.features import DivIcon


# Displaying threat origins as points on the map
for threat in threats:
    
    # Assigning icons based on threat type
    if threat["type"] == "alien":
        icon_html = '<div style="font-size: 24px;">👽</div>'
    elif threat["type"] == "kaiju":
        icon_html = '<div style="font-size: 24px;">🦖</div>'
    elif threat["type"] == "airborne":
        icon_html = '<div style="font-size: 24px;">✈️</div>'
    elif threat["type"] == "orbital":
        icon_html = '<div style="font-size: 24px;">🛰️</div>'
    else:
        icon_html = '<div style="font-size: 24px;">❓</div>'

# Adding markers for each threat origin with appropriate icons
    folium.Marker(
        location=[threat["origin_lat"], threat["origin_lon"]],
        popup=f'{threat["id"]} - {threat["type"]}',
        icon=DivIcon(html=icon_html)
    ).add_to(m)

In [ ]:
m

In [45]:
m.save("World_with_threats.html")

## Milestone 3 - Trajectories

### Tasks: For each threat: compute distination point after a fixed time interval
###                         generate intermediate points
###                         construct a LineString trajectory
###        Plot trajectories on the map
